In [1]:
import random
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
from sklearn.preprocessing import MinMaxScaler
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import numpy as np
import torch
from torch.nn.utils.rnn import pad_sequence
import torchmetrics
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from pytorch_lightning.callbacks import StochasticWeightAveraging

torch.manual_seed(0)
np.random.seed(0)
random.seed(0)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [2]:
%load_ext tensorboard
# %tensorboard --logdir logs

In [3]:
# tensorboard --logdir=tb_logs

In [4]:
#Über r2 per hand berechnen schreiben falls richtig

In [5]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [6]:
df1 = pd.read_csv('/kaggle/input/finalllllllll/Train/Data_processed_punctuality_202212.csv')
df1 = df1.sort_values(by=["journey_id", "current_stop_number"])
df2 = pd.read_csv('/kaggle/input/finalllllllll/Train/Data_processed_punctuality_202207.csv')
df2 = df2.sort_values(by=["journey_id", "current_stop_number"])
df = pd.concat([df1, df2]).reset_index(drop=True)

In [7]:
# df = pd.read_csv('/kaggle/working/all_data.csv')
# df = df.sort_values(by=["journey_id", "current_stop_number"])

In [8]:
# # Directory containing the CSV files
# data_dir = '/kaggle/input/finalllllllll/Train'
# # Lists to store the data from all files
# df_list = []
# for file_name in os.listdir(data_dir):
#     # Only process CSV files
#     if file_name.endswith('.csv'):
#         # Join the directory path and the file name
#         file_path = os.path.join(data_dir, file_name)

#         # Read the data from the file
#         df = pd.read_csv(file_path)

#         # Append the data to the list
#         df_list.append(df)

# # Concatenate all dataframes
# df = pd.concat(df_list, ignore_index=True)
# df = df.sort_values(by=["journey_id", "current_stop_number"])

In [9]:
# df.to_csv('all_data.csv',index=False)

In [10]:
def split_data_by_journey(df, val_size=0.1, test_size=0.2, random_seed=42):
    """Splits a DataFrame into training, validation, and test sets, 
    keeping all rows with the same journey ID in the same set.
    
    Parameters:
    - df: The input DataFrame.
    - val_size: The proportion of journey IDs to include in the validation split (default is 0.1).
    - test_size: The proportion of journey IDs to include in the test split (default is 0.1).
    - random_seed: The seed for the random number generator (default is 42).
    
    Returns:
    - train_df: The training set as a DataFrame.
    - val_df: The validation set as a DataFrame.
    - test_df: The test set as a DataFrame.
    """

    # Get a list of unique journey IDs
    journey_ids = list(df['journey_id'].unique())

    # Shuffle the journey IDs
    np.random.seed(random_seed)
    np.random.shuffle(journey_ids)

    # Split the journey ids into train, val, and test sets
    train_size = int((1 - val_size - test_size) * len(journey_ids))
    val_size = int(val_size * len(journey_ids))
    train_ids = journey_ids[:train_size]
    val_ids = journey_ids[train_size:train_size+val_size]
    test_ids = journey_ids[train_size+val_size:]

    # Create the train, val, and test dataframes based on the split journey ids
    train_df = df[df['journey_id'].isin(train_ids)]
    val_df = df[df['journey_id'].isin(val_ids)]
    test_df = df[df['journey_id'].isin(test_ids)]

    return train_df, val_df, test_df


def generate_sequences(df):
    """
    Generate sequences from the provided DataFrame.
    Each sequence includes a number of consecutive stations in a train's journey. The function randomly selects a
    starting station and includes all previous stations and the starting station as the input, and all subsequent
    stations until the end of the journey as the target. The sequences of station data are returned as the inputs,
    and the sequences of delays at the remaining stations are returned as the targets.

    Args:
    df (pandas.DataFrame): The DataFrame containing the data.

    Returns:
    inputs (list): A list of input sequences. Each sequence is a DataFrame containing the data for a number of
                   consecutive stations.
    targets (list): A list of target sequences. Each sequence is a Series containing the delays at the next
                    stations.
    """
    inputs = []
    targets = []

    # Group the DataFrame by train and date, so that each group represents a journey
    df = df.sort_values(by=['journey_id', 'current_stop_number'])
    grouped = df.groupby(['journey_id'])

    # For each journey
    for _, group in grouped:
        # Skip the journey if it only contains one station
        if len(group) < 10:
            continue

        group = group.reset_index(drop=True)  # Reset the index of the group

        # Select a random starting point for the sequence
        start = np.random.randint(3, len(group) - 5)

        # Get the input sequence and the target sequence
        input_sequence = group.iloc[:start]
        target_sequence = group['arrival_delay_seconds'].iloc[start + 1:].reset_index(drop=True)

        # Append the sequences to the lists
        inputs.append(input_sequence)
        targets.append(target_sequence)

    return inputs, targets


def preprocess_data(train_df, val_df, test_df, cols_to_scale):
    """
    Preprocess the data by scaling specified columns using MinMaxScaler.

    Args:
    - train_df (pd.DataFrame): Training dataframe.
    - val_df (pd.DataFrame): Validation dataframe.
    - test_df (pd.DataFrame): Test dataframe.
    - cols_to_scale (list): List of column names to scale.

    Returns:
    - train_df (pd.DataFrame): Scaled training dataframe.
    - val_df (pd.DataFrame): Scaled validation dataframe.
    - test_df (pd.DataFrame): Scaled test dataframe.
    - scaler_out (MinMaxScaler): Scaler object used for scaling.
    """
    scaler_out = MinMaxScaler()

    # Fit and transform the scaler on the training data
    train_df.loc[:, cols_to_scale] = scaler_out.fit_transform(train_df[cols_to_scale])

    # Transform the validation and test data using the same scaler
    val_df.loc[:, cols_to_scale] = scaler_out.transform(val_df[cols_to_scale])
    test_df.loc[:, cols_to_scale] = scaler_out.transform(test_df[cols_to_scale])

    return train_df, val_df, test_df, scaler_out


def sequences_to_tensors_with_error_info(inputs, targets, columns_drop, num_errors=5):
    """ 
    Convert sequences to tensors and return the errors for a few sequences.
    
    Args:
    - inputs (list): List of input sequences.
    - targets (list): List of target sequences.
    - num_errors (int): Number of errors to print.
    
    Returns:
    - tensor_inputs (list): List of tensor input sequences.
    - tensor_targets (list): List of tensor target sequences.
    - error_count (int): Number of sequences causing errors.
    """
    tensor_inputs = []
    tensor_targets = []
    error_count = 0

    for idx, (inp, tgt) in enumerate(zip(inputs, targets)):
        try:
            input_columns = [col for col in inp.columns if col not in columns_drop]
            tensor_input = torch.tensor(inp[input_columns].values, dtype=torch.float32)
            tensor_target = torch.tensor(tgt.values, dtype=torch.float32).unsqueeze(-1)

            tensor_inputs.append(tensor_input)
            tensor_targets.append(tensor_target)
        except Exception as e:
            error_count += 1
            if error_count <= num_errors:
                print(f"Error with sequence at index '{idx}': {str(e)}")
                print(f"Input sequence:\n{inp}")
                print(f"Target sequence:\n{tgt}\n{'-' * 50}")

    return tensor_inputs, tensor_targets, error_count

In [11]:
# df.info(verbose=True, show_counts=True)

In [12]:
scale_cols = ['current_station_latitude', 'next_station_latitude', 'current_station_longitude',
              'next_station_longitude', 'temperature', 'wind_peak_gust', 'next_station_temperature',
              'next_station_wind_peak_gust', 'median_delay_next_station']
# drop_cols=['journey_id','train_number','current_station_id','next_station_id','unique_journey_id','relation_direction',]
drop_cols = ['journey_id', 'train_number', 'unique_journey_id']
# split data into train and test sets
train_df, val_df, test_df = split_data_by_journey(df=df)
print(f"Train set size: {train_df.shape[0]} rows")
print(f"Test set size: {test_df.shape[0]} rows")
print(f"Validation set size: {val_df.shape[0]} rows")
train_df, val_df, test_df, scaler = preprocess_data(train_df, val_df, test_df, scale_cols)


# 1. Generate sequences using the provided method on the entire dataset
train_inputs, train_targets = generate_sequences(train_df)
val_inputs, val_targets = generate_sequences(val_df)
test_inputs, test_targets = generate_sequences(test_df)


Train set size: 2228732 rows
Test set size: 639430 rows
Validation set size: 319324 rows


/tmp/ipykernel_28/1690434297.py:104: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df.loc[:, cols_to_scale] = scaler_out.fit_transform(train_df[cols_to_scale])
/tmp/ipykernel_28/1690434297.py:107: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val_df.loc[:, cols_to_scale] = scaler_out.transform(val_df[cols_to_scale])
/tmp/ipykernel_28/1690434297.py:108: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

Se

In [13]:
# Convert sequences to tensors for train, validation, and test datasets
tensor_train_inputs, tensor_train_targets, num_errors_train = sequences_to_tensors_with_error_info(train_inputs, train_targets, drop_cols)
tensor_val_inputs, tensor_val_targets, num_errors_val = sequences_to_tensors_with_error_info(val_inputs, val_targets, drop_cols)
tensor_test_inputs, tensor_test_targets, num_errors_test = sequences_to_tensors_with_error_info(test_inputs, test_targets, drop_cols)
len(tensor_train_inputs), len(tensor_val_inputs), len(tensor_test_inputs)
print(f"Number of sequences: {len(tensor_train_inputs)}")
print(f"Number of targets: {len(tensor_train_targets)}")
print(f"First sequence shape: {tensor_train_inputs[0].shape}")
print(f"First target shape: {tensor_train_targets[0].shape}")
print(f"First sequence type: {type(tensor_train_inputs[0])}")
print(f"First target type: {type(tensor_train_targets[0])}")

Number of sequences: 62703
Number of targets: 62703
First sequence shape: torch.Size([3, 31])
First target shape: torch.Size([16, 1])
First sequence type: <class 'torch.Tensor'>
First target type: <class 'torch.Tensor'>


In [14]:
# %load_ext tensorboard
%tensorboard --logdir tb_logs

In [15]:
# Define the collate function
def custom_collate_fn(batch):
    sequences, targets = zip(*batch)

    # Sort sequences by length
    sorted_lengths = sorted([(len(seq), idx) for idx, seq in enumerate(sequences)], reverse=True)
    sorted_indices = [idx for _, idx in sorted_lengths]

    sorted_sequences = [sequences[idx] for idx in sorted_indices]
    sorted_targets = [targets[idx] for idx in sorted_indices]

    lengths = [len(seq) for seq in sorted_sequences]

    # Pad sequences
    padded_sequences = pad_sequence(sorted_sequences, batch_first=True)
    padded_targets = pad_sequence(sorted_targets, batch_first=True)

    return padded_sequences, padded_targets, lengths, len(sequences)


class CustomDataset(Dataset):
    def __init__(self, sequences, targets):
        self.sequences = sequences
        self.targets = targets

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, index):
        return self.sequences[index], self.targets[index]



In [ ]:

# Specify the directory and experiment name
logger = TensorBoardLogger("tb_logs", name="Seq2SeqGRU")


class Encoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, dropout_p):
        super(Encoder, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.gru = nn.GRU(input_dim, hidden_dim, num_layers, batch_first=True, dropout=dropout_p)

    def forward(self, x, lengths):
        packed_seq = pack_padded_sequence(x, lengths, batch_first=True)
        packed_output, h_n = self.gru(packed_seq)
        output, _ = pad_packed_sequence(packed_output, batch_first=True)
        return output, h_n


class Decoder(nn.Module):
    def __init__(self, output_dim, hidden_dim, num_layers, dropout_p):
        super(Decoder, self).__init__()
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.gru = nn.GRU(output_dim, hidden_dim, num_layers, batch_first=True, dropout=dropout_p)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x, encoder_hidden):
        output, hidden = self.gru(x, encoder_hidden)
        output = self.fc(output)
        return output, hidden


class Seq2SeqGRU(pl.LightningModule):
    def __init__(self, input_dim, output_dim, hidden_dim, num_layers, dropout_p, learning_rate, teacher_forcing_ratio,
                 log_grad_every=1):
        super(Seq2SeqGRU, self).__init__()
        self.encoder = Encoder(input_dim, hidden_dim, num_layers, dropout_p)
        self.decoder = Decoder(output_dim, hidden_dim, num_layers, dropout_p)
        self.learning_rate = learning_rate
        self.teacher_forcing_ratio = teacher_forcing_ratio
        self.log_grad_every = log_grad_every

        self.train_r2 = torchmetrics.R2Score()
        self.val_r2 = torchmetrics.R2Score()
        self.test_r2 = torchmetrics.R2Score()
        
        self.train_mse = torchmetrics.MeanSquaredError()
        self.val_mse = torchmetrics.MeanSquaredError()
        self.test_mse = torchmetrics.MeanSquaredError()
        
        self.train_rmse = torchmetrics.MeanSquaredError(squared=False)
        self.val_rmse = torchmetrics.MeanSquaredError(squared=False)
        self.test_rmse = torchmetrics.MeanSquaredError(squared=False)



    def calculate_metrics(self, output, target, mode="train"):
        """
        Calculate metrics and log them.
        """
        # Ensure both output and target are reshaped to 2D tensors
        output_2d = output.squeeze(-1)
        target_2d = target.squeeze(-1)
    
        # Check for dimension mismatch between output and target
        if output_2d.shape != target_2d.shape:
            raise ValueError(f"Shape mismatch: output {output_2d.shape} vs target {target_2d.shape}")
    
        loss = F.l1_loss(output, target)
        mse = getattr(self, f"{mode}_mse")(output_2d, target_2d)
    
        # Manual R^2 calculation
        ss_res = torch.sum((target - output) ** 2)
        ss_tot = torch.sum((target - torch.mean(target)) ** 2)
        r2 = 1 - ss_res / ss_tot
    
        rmse = getattr(self, f"{mode}_rmse")(output_2d, target_2d)
    
        metrics = {
            f'{mode}_loss': loss,
            f'{mode}_mse': mse,
            f'{mode}_rmse': rmse,
            f'{mode}_r2': r2
        }
        return metrics

    def forward(self, source, target, lengths):
        # Encoder part
        encoder_output, encoder_hidden = self.encoder(source, lengths)

        # Decoder part
        target_length = target.shape[1]
        outputs = torch.zeros(target.shape).to(device)
        decoder_input = target[:, 0, :].unsqueeze(1)

        for t in range(1, target_length):
            decoder_output, decoder_hidden = self.decoder(decoder_input, encoder_hidden)
            outputs[:, t, :] = decoder_output.squeeze(1)
            decoder_input = target[:, t, :].unsqueeze(
                1) if torch.rand(1).item() < self.teacher_forcing_ratio else decoder_output

        return outputs

    def training_step(self, batch, batch_idx):
        source, target, lengths, batch_size = batch
        output = self.forward(source, target, lengths)
        metrics = self.calculate_metrics(output, target, mode="train")
        self.log_dict(metrics, on_step=True, on_epoch=True, batch_size=batch_size)
        return {'loss': metrics['train_loss']}


    def validation_step(self, batch, batch_idx):
        source, target, lengths, batch_size = batch
        output = self.forward(source, target, lengths)
        metrics = self.calculate_metrics(output, target, mode="val")
        self.log_dict(metrics, on_step=True, on_epoch=True, batch_size=batch_size)
    
        return {'val_loss': metrics['val_loss']}

    def test_step(self, batch, batch_idx):
        source, target, lengths, batch_size = batch
        output = self.forward(source, target, lengths)
        metrics = self.calculate_metrics(output, target, mode="test")
        self.log_dict(metrics, on_step=True, on_epoch=True, batch_size=batch_size)
        return {'test_loss': metrics['test_loss']}

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr=self.learning_rate)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min')
        return {
            'optimizer': optimizer,
            'lr_scheduler': scheduler,
            'monitor': 'val_loss'
        }

    def on_after_backward(self):
        if self.current_epoch % self.log_grad_every == 0:  # Log every `log_grad_every` epochs
            for name, param in self.named_parameters():
                if param.grad is not None:
                    self.logger.experiment.add_histogram(name + '_grad', param.grad, self.current_epoch)


input_dim = len(tensor_train_inputs[0][0])  # Feature length
output_dim = 1  # Predicting 'arrival_delay_seconds'
hidden_dim = 512  # Hidden layer dimension
num_layers = 2  # Number of GRU layers
dropout_p = 0.536079240699101  # Dropout probability
batch_size = 256  # Batch size for DataLoader
epochs = 70  # Number of training epochs
learning_rate = 0.0009575376018764724  # Learning rate for optimizer
teacher_forcing_ratio = 0.7338010785815597  # Probability of using teacher forcing during training

# Create DataLoaders
# Create your custom datasets
train_dataset = CustomDataset(tensor_train_inputs, tensor_train_targets)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True, collate_fn=custom_collate_fn)

val_dataset = CustomDataset(tensor_val_inputs, tensor_val_targets)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True, collate_fn=custom_collate_fn)

test_dataset = CustomDataset(tensor_test_inputs, tensor_test_targets)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True, collate_fn=custom_collate_fn)


# Model
model = Seq2SeqGRU(input_dim, output_dim, hidden_dim, num_layers, dropout_p, learning_rate, teacher_forcing_ratio)
# Define the StochasticWeightAveraging callback
# swa = StochasticWeightAveraging(swa_epoch_start=0.8)
# Early stopping callback
early_stop_callback = EarlyStopping(
    monitor='val_loss',
    patience=10,
    verbose=False,
    mode='min'
)
# Training
# trainer = pl.Trainer(max_epochs=epochs, accelerator="auto", callbacks=[early_stop_callback,StochasticWeightAveraging], log_every_n_steps=1,
#                      logger=logger)
trainer = pl.Trainer(max_epochs=epochs, accelerator="auto", callbacks=[early_stop_callback], log_every_n_steps=25,
                     logger=logger)


# trainer = pl.Trainer(max_epochs=epochs, accelerator="auto", log_every_n_steps=1)
trainer.fit(model, train_loader, val_loader)

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [ ]:
%tensorboard --logdir tb_logs


In [ ]:
trainer.test(dataloaders=test_loader, ckpt_path='best')

In [ ]:
df3 = pd.read_csv('/kaggle/input/finalllllllll/Train/Data_processed_punctuality_202212.csv')
df3 = df3.sort_values(by=["journey_id", "current_stop_number"])
df3= df.head(100_000)

train_df, val_df, test_df = split_data_by_journey(df=df3)
train_df, val_df, test_df, scaler = preprocess_data(train_df, val_df, test_df, scale_cols)
train_inputs, train_targets = generate_sequences(train_df)
train_dataset = CustomDataset(tensor_train_inputs, tensor_train_targets)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True, collate_fn=custom_collate_fn)
val_dataset = CustomDataset(tensor_val_inputs, tensor_val_targets)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True, collate_fn=custom_collate_fn)
import optuna

def objective(trial):
    # Hyperparameters to be optimized
    hidden_dim = trial.suggest_int('hidden_dim', 64, 1024)
    num_layers = trial.suggest_int('num_layers', 2, 6)
    dropout_p = trial.suggest_float('dropout_p', 0.1, 0.7)
    learning_rate = trial.suggest_float('lr', 1e-4, 1e-2, log=True)
    teacher_forcing_ratio = trial.suggest_float('teacher_forcing_ratio', 0.3, 0.8)

    # Model
    model = Seq2SeqGRU(input_dim, output_dim, hidden_dim, num_layers, dropout_p, learning_rate, teacher_forcing_ratio)


    trainer = (pl.Trainer
               (max_epochs=20, 
                accelerator="auto", 
                callbacks=[early_stop_callback], 
                log_every_n_steps=5,
                logger=logger))

    trainer.fit(model, train_loader, val_loader)

    # Return the best validation loss achieved by the model
    val_loss = trainer.callback_metrics["val_loss"].item()
    return val_loss


In [ ]:
study = optuna.create_study(direction='minimize')  # we want to minimize the validation loss
study.optimize(objective, n_trials=10)  # number of trials


In [ ]:
print("Number of finished trials: ", len(study.trials))
print("Best trial:")
trial = study.best_trial

print("Value: ", trial.value)
print("Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


In [ ]:
torch.save(model, 'model.pth')


In [ ]:
def objective(trial):
    # Hyperparameters to be optimized
    hidden_dim = trial.suggest_int('hidden_dim', 64, 1024)
    num_layers = trial.suggest_int('num_layers', 2, 6)
    dropout_p = trial.suggest_float('dropout_p', 0.1, 0.7)
    learning_rate = trial.suggest_float('lr', 1e-4, 1e-2, log=True)
    teacher_forcing_ratio = trial.suggest_float('teacher_forcing_ratio', 0.3, 0.8)

    # Model
    model = Seq2SeqGRU(input_dim, output_dim, hidden_dim, num_layers, dropout_p, learning_rate, teacher_forcing_ratio)

    trainer = (pl.Trainer
               (max_epochs=20, 
                accelerator="auto", 
                callbacks=[early_stop_callback], 
                log_every_n_steps=5,
                logger=logger))

    trainer.fit(model, train_loader, val_loader)

    val_loss = trainer.callback_metrics["val_loss"].item()
    return val_loss
study = optuna.create_study(direction='minimize')  
study.optimize(objective, n_trials=10) 
print("Number of finished trials: ", len(study.trials))
print("Best trial:")
trial = study.best_trial

print("Value: ", trial.value)
print("Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

In [ ]:
# # Switch the model to evaluation mode
# model.eval()
# 
# # List to store all predictions
# all_predictions = []
# 
# # List to store actual targets
# all_targets = []
# 
# # Iterate over all sequences
# for source, target, lengths in test_loader:
#     # Move data to the device
#     source, target = source.to(device), target.to(device)
# 
#     # Generate prediction
#     with torch.no_grad():
#         output = model(source, target, lengths, 0)  # 0 teacher forcing ratio for evaluation
# 
#     # Move prediction to CPU and convert to numpy array
#     prediction = output.cpu().numpy()
#     actual = target.cpu().numpy()
# 
#     # Store prediction
#     all_predictions.append(prediction)
#     all_targets.append(actual)
# 
# # Concatenate all predictions into a single numpy array
# all_predictions = np.concatenate(all_predictions)
# all_targets = np.concatenate(all_targets)
# # Assuming df_test is your original test dataframe and "target_column" is the name of your target column
# df_predictions = test_df.copy()
# df_predictions["arrival_delay_seconds"] = all_predictions.reshape(-1)  # Flatten the predictions array
# 
# # Now df_predictions includes both actual and predicted target values
# print(df_predictions)
# df_predictions.to_csv('predictions.csv', index=False)
